In [1]:
%load_ext autoreload
%autoreload 2
from nlp_utils import *
from data_utils import *
from KeywordExtractor import *
from gensim.models import Word2Vec
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import nltk
from nltk.corpus import wordnet
import json

methods = {1: "degree_centrality", 2: "closeness_centrality", 3: "betweenness_centrality", 4:"eigenvector_centrality", 5:"pagerank"} #, \
           #6:"katz_centrality"} #, 7: "hits"}

In [2]:
# WORD EMBEDDINGS
from gensim.models import Word2Vec, KeyedVectors
import gensim.downloader as api
# Download a pre-trained Word2Vec model (you can choose other models as well)
model_name = "glove-wiki-gigaword-50"

# Download and load the model (this might take a while to download)
# model = api.load(model_name)
# model.save('data/model.model')

In [10]:
ws = 10
json_file_path = f'data/mean_metrics_{ws}.json'

# Open the file and load its contents into a dictionary
with open(json_file_path, 'r') as file:
    json_data = json.load(file)
new_dict = {'Precision': [], 'Recall': [], "F-score": []}
for key in json_data.keys():
    if key[-1] == "p":
        new_dict['Precision'].append(round(json_data[key]*100, 2))
    elif key[-1] == "r":
        new_dict['Recall'].append(round(json_data[key]*100, 2))
    elif key[-1] == "f":
        new_dict['F-score'].append(round(json_data[key]*100, 2))
df = pd.DataFrame(new_dict)
df.to_csv(f'data/m_{ws}.csv')

In [36]:
true_labels = ['Worlds', 'b', 'c', 'd', 'e']
predicted_labels = ['world_war_2', 'c', 'j', 'k', 'e', 'op']
p, r, f = get_prf(true_labels, predicted_labels)
print(f'Precision: {p:.4f} \nRecall: {r:.4f} \nF-score: {f:.4f}')

Precision: 0.5000 
Recall: 0.6000 
F-score: 0.5455


In [4]:
path_to_file = 'data/CS_unique.csv'

#joined, abstracts, titles, keywords = get_data(path_to_file, version="CS")
metrics_dict = make_keyword_metrics(methods, path_to_file, window_size=10)

Finished 1. abstract
Failed on abstract: 6, 
 error: (PowerIterationFailedConvergence(...), 'power iteration failed to converge within 100 iterations')
Failed on abstract: 18, 
 error: (PowerIterationFailedConvergence(...), 'power iteration failed to converge within 100 iterations')
Failed on abstract: 35, 
 error: (PowerIterationFailedConvergence(...), 'power iteration failed to converge within 100 iterations')
Failed on abstract: 63, 
 error: (PowerIterationFailedConvergence(...), 'power iteration failed to converge within 100 iterations')
Failed on abstract: 158, 
 error: (PowerIterationFailedConvergence(...), 'power iteration failed to converge within 100 iterations')
Finished 201. abstract
Failed on abstract: 242, 
 error: (PowerIterationFailedConvergence(...), 'power iteration failed to converge within 100 iterations')
Failed on abstract: 255, 
 error: (PowerIterationFailedConvergence(...), 'power iteration failed to converge within 100 iterations')
Failed on abstract: 296, 
 err

In [ ]:
abstract = "The island country of Japan has developed into a great economy after World War 2. \
The Japan sea is a source of fish. Sushi is a famous fish and rice food."
tokens, sentences = prune_text(abstract)
co, index_dict = get_co(sentences, representation='matrix')
print(index_dict)
print(co)

In [6]:
# abstract = load_abstract('ex1')
# joined, abstracts, titles, keywords = get_data('data/CS_papers.csv', version="CS")
# abstracts, keywords = get_data('data/inspec_papers.csv', version="inspec")
# abstract = joined[12]
abstract = "The island country of Japan has developed into a great economy after World War 2. \
The Japan sea is a source of fish. Sushi is a famous fish and rice food."
ke = KeywordExtractor(abstract=abstract)
print(abstract)
print(ke.tokens)
print(ke.sentences)
print(ke.co)
ke.graph.nodes

The island country of Japan has developed into a great economy after World War 2. The Japan sea is a source of fish. Sushi is a famous fish and rice food.
['island', 'country', 'japan', 'great', 'economy', 'world_war_2', 'japan', 'sea', 'source', 'fish', 'sushi', 'famous', 'fish', 'rice', 'food']
[['island', 'country', 'japan', 'great', 'economy', 'world_war_2'], ['japan', 'sea', 'source', 'fish'], ['sushi', 'famous', 'fish', 'rice', 'food']]
{('country', 'island'): 1, ('island', 'japan'): 1, ('country', 'japan'): 2, ('country', 'great'): 1, ('great', 'japan'): 2, ('economy', 'japan'): 1, ('economy', 'great'): 2, ('great', 'world_war_2'): 1, ('economy', 'world_war_2'): 1, ('japan', 'sea'): 1, ('japan', 'source'): 1, ('sea', 'source'): 2, ('fish', 'sea'): 1, ('fish', 'source'): 1, ('famous', 'sushi'): 1, ('fish', 'sushi'): 1, ('famous', 'fish'): 2, ('famous', 'rice'): 1, ('fish', 'rice'): 2, ('fish', 'food'): 1, ('food', 'rice'): 1}


NodeView(('food', 'sushi', 'rice', 'country', 'world_war_2', 'famous', 'island', 'fish', 'sea', 'source', 'economy', 'japan', 'great'))

Testing third party keyword extractors

In [ ]:
from gensim.summarization import keywords

# Extract keywords using TextRank
extracted_keywords = keywords(abstract, lemmatize=True)

# Print the extracted keywords
print(extracted_keywords)

In [ ]:
import spacy
import pytextrank

# example text
text = abstract

# load a spaCy model, depending on language, scale, etc.
nlp = spacy.load("en_core_web_sm")

# add PyTextRank to the spaCy pipeline
nlp.add_pipe("textrank")
doc = nlp(text)

# examine the top-ranked phrases in the document
#print(f"Ground truth {keywords[0]}")
for phrase in doc._.phrases:
    print(phrase.text)
    print(phrase.rank, phrase.count)
    print(phrase.chunks)

In [10]:
print(len(ke.unique_tokens))
labels = nx.get_edge_attributes(ke.graph,'weight')
# ke.add_we_weights()
#labels = nx.get_edge_attributes(ke.graph,'weight')

# print(ke.graph.edges)
print(labels)
methods = {1: "degree_centrality", 2: "closeness_centrality", 3: "betweenness_centrality", 4:"eigenvector_centrality", 5:"pagerank", \
           6:"katz_centrality"} #, 7: "hits"}
keyword_dict = ke.order_nodes(method=methods[1], to_print=False)
predicted_keywords = list(keyword_dict.keys())[:5]
print(predicted_keywords)
# make_metrics_csv(ke, methods)
# print(nx.shortest_path(ke.graph, source="rice", target="japan", weight="weight"))
# print(nx.shortest_path_length(ke.graph, source="world_war_2", target="japan", weight="weight"))
# ke.visualize_graph()

13
{('food', 'fish'): 1, ('food', 'rice'): 1, ('sushi', 'famous'): 1, ('sushi', 'fish'): 1, ('rice', 'famous'): 1, ('rice', 'fish'): 2, ('country', 'island'): 1, ('country', 'japan'): 2, ('country', 'great'): 1, ('world_war_2', 'great'): 1, ('world_war_2', 'economy'): 1, ('famous', 'fish'): 2, ('island', 'japan'): 1, ('fish', 'sea'): 1, ('fish', 'source'): 1, ('sea', 'japan'): 1, ('sea', 'source'): 2, ('source', 'japan'): 1, ('economy', 'japan'): 1, ('economy', 'great'): 2, ('japan', 'great'): 2}
Method selected: degree_centrality
['fish', 'japan', 'great', 'rice', 'country']


In [18]:
# Create the graph
G = nx.Graph()
G.add_node('A')
G.add_node('B')
G.add_node('C')
G.add_node('D')
G.add_node('E')
# Add edges
G.add_edge('A', 'B', weight=1)
G.add_edge('A', 'C', weight=3)
G.add_edge('B', 'E', weight=2)
G.add_edge('C', 'D', weight=2)
G.add_edge('D', 'E', weight=1)
G.add_edge('A', 'D', weight=1)

# Calculate degree centrality
d = 1

degree_centrality = nx.katz_centrality(G, weight="weight", alpha = 0.20, beta=0.1)
degree_centrality = nx.betweenness_centrality(G, weight="weight")

# Print degree centrality for each node
for node, centrality in degree_centrality.items():
    print(f'Centrality of node {node}: {centrality:.5f}')

A = nx.adjacency_matrix(G).toarray()
I = np.eye(5)
alpha = 0.2
C = (np.linalg.inv(I - alpha * A)) @ np.ones(5)
#C = (np.linalg.inv(I - alpha * A.T) - I) @ np.ones(5)
print(C)
print(C / np.sqrt(np.sum(C**2)))

#print(A)
# A = A / A.sum(axis=0, keepdims=True)
# print(A)
# eigenvalues, eigenvectors = np.linalg.eig(A)
# print(eigenvalues)
# dominant_index = np.argmax(eigenvalues)
# print(dominant_index)
# dominant_eigenvector = eigenvectors[:, dominant_index]
# print(dominant_eigenvector)
# print(dominant_eigenvector/np.sum(dominant_eigenvector))

# I = np.eye(5)
# R = (1-d)/5 * np.linalg.inv(I - d*np.transpose(A)) @ np.ones(5) 
# print(f"final: {R}")
# Draw the graph
# pos = nx.spring_layout(G)
# nx.draw(G, pos, with_labels=True, font_weight='bold', node_color='skyblue', node_size=800)
# edge_labels = nx.get_edge_attributes(G, 'weight')
# nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels)
# plt.show()

Centrality of node A: 0.33333
Centrality of node B: 0.00000
Centrality of node C: 0.00000
Centrality of node D: 0.50000
Centrality of node E: 0.00000
[8.99852725 4.51399116 9.35935199 7.4005891  4.28571429]
[0.55584268 0.27883107 0.57813097 0.45713739 0.26473031]


Testing word embeddings

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

model = KeyedVectors.load('data/model.model')
a = get_word_em("nlp")
b = get_word_em("apple")
print(model.similarity("nlp", "apple"))
print(cosine_similarity(a.reshape(1, -1), b.reshape(1, -1))[0])
similar_words = model.similar_by_vector(a, topn=5)
print("\nWords similar to:")
for word, score in similar_words:
    print(f"{word}: {score}")

In [2]:
model = KeyedVectors.load('data/model.model')
# Find the vector representation of a word
word_vector = model["apple"]
print("Vector representation of 'apple':")
print(word_vector)

# Find similar words
similar_words = model.most_similar("apple", topn=5)
print("\nWords similar to 'apple':")
for word, score in similar_words:
    print(f"{word}: {score}")

# Perform vector arithmetic (e.g., king - man + woman = queen)
result = model.most_similar(positive=["king", "woman"], negative=["man"], topn=1)
print("\n'king' - 'man' + 'woman' =")
for word, score in result:
    print(f"{word}: {score}")

# Assuming you have already loaded a pretrained Word2Vec model
word1 = "learning"
word2 = "island"

# Calculate the similarity between two words
similarity_score = model.similarity(word1, word2)

print(f"Similarity between '{word1}' and '{word2}': {similarity_score:.2f}")


Vector representation of 'apple':
[ 0.52042  -0.8314    0.49961   1.2893    0.1151    0.057521 -1.3753
 -0.97313   0.18346   0.47672  -0.15112   0.35532   0.25912  -0.77857
  0.52181   0.47695  -1.4251    0.858     0.59821  -1.0903    0.33574
 -0.60891   0.41742   0.21569  -0.07417  -0.5822   -0.4502    0.17253
  0.16448  -0.38413   2.3283   -0.66682  -0.58181   0.74389   0.095015
 -0.47865  -0.84591   0.38704   0.23693  -1.5523    0.64802  -0.16521
 -1.4719   -0.16224   0.79857   0.97391   0.40027  -0.21912  -0.30938
  0.26581 ]

Words similar to 'apple':
blackberry: 0.7543067932128906
chips: 0.7438643574714661
iphone: 0.7429664134979248
microsoft: 0.7334205508232117
ipad: 0.7331036925315857

'king' - 'man' + 'woman' =
queen: 0.8523604273796082
Similarity between 'learning' and 'island': 0.25
